In [1]:
!pip install autogluon
!git clone https://github.com/priorlabs/tabpfn-extensions.git
!pip install -e tabpfn-extensions
!pip install tabpfn
!pip install tabpfn_extensions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━

In [2]:
# prompt: import pandas and basic machine learning models for regression

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR


from sklearn.model_selection import train_test_split

import itertools
import random

import torch
import random
import numpy as np

import os
import joblib

import matplotlib.pyplot as plt

from tabpfn import TabPFNRegressor
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

# from sklearn.metrics import mean_absolute_percentage_error
# from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import AutoTabPFNRegressor

In [3]:
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import mean_absolute_percentage_error

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
df = pd.read_csv('train.csv')

In [11]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [12]:
# display(df.head())
# display(df.info())
# display(df.describe())

## Property 1

In [13]:
df_set1 = df_train[fraction_cols + property_tn  + [target]]

In [ ]:
# augment_data(df_set1, 3, 1, shuffle=0)

## Property 1

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
tn = 1
fn = tn

property_tn = [f'Component{i+1}_Property{fn}' for i in range(5)]
fraction_cols = [f'Component{i+1}_fraction' for i in range(5)]
target_cols = [f"BlendProperty{i}" for i in range(1, 11)]
components = ['Component1', 'Component2', 'Component3', 'Component4', 'Component5']
properties = [f'Property{i}' for i in range(1, 11)]
target = target_cols[tn-1]

In [ ]:
# Generate polynomial features (degree=2 by default)
# Set interaction_only=True to include only interaction features
X = df_train[fraction_cols + property_tn ]
poly1 = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
X1_poly = poly1.fit_transform(X)

# Create DataFrame from transformed features
# df_p1 = pd.DataFrame(X1_poly, columns=poly1.get_feature_names(fraction_cols + property_tn ) )


# Initialize and train a Linear Regression model on the polynomial features
linear_model_poly = LinearRegression()
linear_model_poly.fit(X1_poly, df_train[target_cols[tn-1]] )

print("Linear Regression model fitted successfully on polynomial features.")


selected_cols = fraction_cols + property_tn

# Prepare the test data for prediction
X_test_poly = poly1.transform(df_test[selected_cols])


# Make predictions using the trained polynomial model
predictions_poly = linear_model_poly.predict(X_test_poly)
y_test1 = df_test[target_cols[tn-1]]


# Calculate the Mean Absolute Percentage Error (MAPE) on the test set
mape_poly = mean_absolute_percentage_error(y_test1, predictions_poly)

print(f"Mean Absolute Percentage Error (MAPE) on test set with polynomial features: {mape_poly:.4f}")

Linear Regression model fitted successfully on polynomial features.
Mean Absolute Percentage Error (MAPE) on test set with polynomial features: 0.0053


In [ ]:
import joblib
import os

# Define the base directory in Google Drive
home = '/content/drive/MyDrive/Projects/Shell/App/Models'

# Ensure the directory exists
# os.makedirs(home, exist_ok=True)

# Define the filename for the saved model including target_1
model_filename = os.path.join(home, 'linear_model_poly_target_1.joblib')

# Save the trained model to the file
joblib.dump(linear_model_poly, model_filename)

print(f"Model saved successfully to {model_filename}")

Model saved successfully to /content/drive/MyDrive/Projects/Shell/App/Models/linear_model_poly_target_1.joblib


In [ ]:

# Define the base directory in Google Drive
home = '/content/drive/MyDrive/Projects/Shell/App/Models'
os.makedirs(home, exist_ok=True)
poly1_filename = os.path.join(home, 'poly1_features.joblib')

# Assuming poly1 object has been created in a previous cell
# Make sure to run the cell that creates poly1 before running this cell
if 'poly1' in locals():
    joblib.dump(poly1, poly1_filename)
    print(f"PolynomialFeatures object poly1 saved successfully to {poly1_filename}")
else:
    print("Error: poly1 object not found. Please run the cell that creates poly1 first.")

PolynomialFeatures object poly1 saved successfully to /content/drive/MyDrive/Projects/Shell/App/Models/poly1_features.joblib


In [ ]:
# Display the coefficients
print("Coefficients:")
display(linear_model_poly.coef_)

# Display the intercept
print("\nIntercept:")
display(linear_model_poly.intercept_)

Coefficients:


array([-2.74091865e+00, -4.14944376e+00, -1.76145076e+00,  6.25103091e-01,
        8.02671008e+00,  1.61079706e-01,  1.16713290e-01,  1.98991935e-01,
        9.92194616e-02,  2.37496892e-01, -6.85234957e-01, -1.72253979e+00,
       -1.12556398e+00, -5.29002768e-01,  1.32142284e+00,  8.05392752e-01,
       -1.16698850e-01, -1.98988809e-01, -9.92124404e-02, -2.37497437e-01,
       -1.03733344e+00, -1.47769742e+00, -8.81099109e-01,  9.69225998e-01,
       -1.61078959e-01,  5.83564549e-01, -1.98996000e-01, -9.92182063e-02,
       -2.37495347e-01, -4.40360251e-01, -2.84087984e-01,  1.56625888e+00,
       -1.61077938e-01, -1.16707281e-01,  9.94960790e-01, -9.92137959e-02,
       -2.37495910e-01,  1.56248780e-01,  2.16304417e+00, -1.61083838e-01,
       -1.16719900e-01, -1.98993976e-01,  4.96102062e-01, -2.37498668e-01,
        2.00675819e+00, -1.61072310e-01, -1.16725228e-01, -1.98990070e-01,
       -9.92381581e-02,  1.18748425e+00,  8.02421098e-07,  6.35954469e-07,
        1.39572212e-06,  


Intercept:


np.float64(0.5310235898634373)

In [ ]:
# prompt: create an ml model using X_poly and y_tain



In [ ]:
# Get the feature names after polynomial transformation
feature_names = poly1.get_feature_names_out(X_train_1_augmented.columns)

# Get the coefficients from the trained linear model
coefficients = linear_model_poly.coef_

# Create a pandas Series mapping feature names to coefficients
coefficients_series = pd.Series(coefficients, index=feature_names)

# Display the series in a scrollable table format
print("Polynomial Feature Coefficients:")
display(coefficients_series)

Polynomial Feature Coefficients:


,0
Component1_fraction,0.754109
Component2_fraction,-0.332503
Component3_fraction,0.760428
Component4_fraction,0.632219
Component5_fraction,-1.814253
Component1_Property1,0.140550
Component2_Property1,0.145206
Component3_Property1,0.173276
Component4_Property1,0.125836
Component5_Property1,0.152889


##  Property 2

In [ ]:
# Set up for Property 2
tn = 2
fn = tn

property_tn = [f'Component{i+1}_Property{fn}' for i in range(5)]
fraction_cols = [f'Component{i+1}_fraction' for i in range(5)]
target_cols = [f"BlendProperty{i}" for i in range(1, 11)]
components = ['Component1', 'Component2', 'Component3', 'Component4', 'Component5']
properties = [f'Property{i}' for i in range(1, 11)]
target = target_cols[tn-1]

# Select Data (using df_train as augmentation is skipped)
X = df_train[fraction_cols + property_tn ]

# Polynomial Features (using the same settings as Property 1 example)
poly2 = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
X2_poly = poly2.fit_transform(X)

# Initialize and train a Linear Regression model on the polynomial features for Property 2
linear_model_poly_prop2 = LinearRegression()
linear_model_poly_prop2.fit(X2_poly, df_train[target] )

print(f"Linear Regression model fitted successfully on polynomial features for {target}.")

selected_cols = fraction_cols + property_tn

# Prepare the test data for prediction
X_test_poly_prop2 = poly2.transform(df_test[selected_cols])


# Make predictions using the trained polynomial model for Property 2
predictions_poly_prop2 = linear_model_poly_prop2.predict(X_test_poly_prop2)
y_test2 = df_test[target]


# Calculate the Mean Absolute Percentage Error (MAPE) on the test set for Property 2
mape_poly_prop2 = mean_absolute_percentage_error(y_test2, predictions_poly_prop2)

print(f"Mean Absolute Percentage Error (MAPE) on test set with polynomial features for {target}: {mape_poly_prop2:.4f}")



Linear Regression model fitted successfully on polynomial features for BlendProperty2.
Mean Absolute Percentage Error (MAPE) on test set with polynomial features for BlendProperty2: 0.0608


In [ ]:
# Save the model for Property 2


# Define the base directory in Google Drive
home = '/content/drive/MyDrive/Projects/Shell/App/Models'

# Ensure the directory exists
os.makedirs(home, exist_ok=True)

# Define the filename for the saved model including target_2
model_filename_prop2 = os.path.join(home, 'linear_model_poly_target_2.joblib')

# Save the trained model to the file
joblib.dump(linear_model_poly_prop2, model_filename_prop2)

print(f"Model for {target} saved successfully to {model_filename_prop2}")

Model for BlendProperty2 saved successfully to /content/drive/MyDrive/Projects/Shell/App/Models/linear_model_poly_target_2.joblib


In [ ]:

# Define the base directory in Google Drive
home = '/content/drive/MyDrive/Projects/Shell/App/Models'
os.makedirs(home, exist_ok=True)
poly2_filename = os.path.join(home, 'poly2_features.joblib')

# Assuming poly2 object has been created in a previous cell
# Make sure to run the cell that creates poly2 before running this cell
if 'poly2' in locals():
    joblib.dump(poly2, poly2_filename)
    print(f"PolynomialFeatures object poly2 saved successfully to {poly2_filename}")
else:
    print("Error: poly2 object not found. Please run the cell that creates poly2 first.")

PolynomialFeatures object poly2 saved successfully to /content/drive/MyDrive/Projects/Shell/App/Models/poly2_features.joblib


## Property 6

In [ ]:
# Set up for Property 6
tn = 6
fn = tn

property_tn = [f'Component{i+1}_Property{fn}' for i in range(5)]
fraction_cols = [f'Component{i+1}_fraction' for i in range(5)]
target_cols = [f"BlendProperty{i}" for i in range(1, 11)]
components = ['Component1', 'Component2', 'Component3', 'Component4', 'Component5']
properties = [f'Property{i}' for i in range(1, 11)]
target = target_cols[tn-1]

# Select Data (using df_train)
X = df_train[fraction_cols + property_tn ]

# Polynomial Features (using the same settings as Property 1 and 2 examples)
poly6 = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
X6_poly = poly6.fit_transform(X)

# Initialize and train a Linear Regression model on the polynomial features for Property 6
linear_model_poly_prop6 = LinearRegression()
linear_model_poly_prop6.fit(X6_poly, df_train[target] )

print(f"Linear Regression model fitted successfully on polynomial features for {target}.")

selected_cols = fraction_cols + property_tn

# Prepare the test data for prediction
X_test_poly_prop6 = poly6.transform(df_test[selected_cols])


# Make predictions using the trained polynomial model for Property 6
predictions_poly_prop6 = linear_model_poly_prop6.predict(X_test_poly_prop6)
y_test6 = df_test[target]


# Calculate the Mean Absolute Percentage Error (MAPE) on the test set for Property 6
mape_poly_prop6 = mean_absolute_percentage_error(y_test6, predictions_poly_prop6)

print(f"Mean Absolute Percentage Error (MAPE) on test set with polynomial features for {target}: {mape_poly_prop6:.4f}")



# Define the base directory in Google Drive
home = '/content/drive/MyDrive/Projects/Shell/App/Models'

# Ensure the directory exists
os.makedirs(home, exist_ok=True)

# Define the filename for the saved model including target_6
model_filename_prop6 = os.path.join(home, 'linear_model_poly_target_6.joblib')

# Save the trained model to the file
joblib.dump(linear_model_poly_prop6, model_filename_prop6)

print(f"Model for {target} saved successfully to {model_filename_prop6}")

# Save the poly6 object
poly6_filename = os.path.join(home, 'poly6_features.joblib')
joblib.dump(poly6, poly6_filename)
print(f"PolynomialFeatures object poly6 saved successfully to {poly6_filename}")

Linear Regression model fitted successfully on polynomial features for BlendProperty6.
Mean Absolute Percentage Error (MAPE) on test set with polynomial features for BlendProperty6: 0.0006
Model for BlendProperty6 saved successfully to /content/drive/MyDrive/Projects/Shell/App/Models/linear_model_poly_target_6.joblib
PolynomialFeatures object poly6 saved successfully to /content/drive/MyDrive/Projects/Shell/App/Models/poly6_features.joblib


## Property 10

In [ ]:
# Set up for Property 10
tn = 10
fn = tn

property_tn = [f'Component{i+1}_Property{fn}' for i in range(5)]
fraction_cols = [f'Component{i+1}_fraction' for i in range(5)]
target_cols = [f"BlendProperty{i}" for i in range(1, 11)]
components = ['Component1', 'Component2', 'Component3', 'Component4', 'Component5']
properties = [f'Property{i}' for i in range(1, 11)]
target = target_cols[tn-1]

# Select Data (using df_train)
X = df_train[fraction_cols + property_tn ]

# Polynomial Features (using the same settings as previous properties)
poly10 = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
X10_poly = poly10.fit_transform(X)

# Initialize and train a Linear Regression model on the polynomial features for Property 10
linear_model_poly_prop10 = LinearRegression()
linear_model_poly_prop10.fit(X10_poly, df_train[target] )

print(f"Linear Regression model fitted successfully on polynomial features for {target}.")

selected_cols = fraction_cols + property_tn

# Prepare the test data for prediction
X_test_poly_prop10 = poly10.transform(df_test[selected_cols])


# Make predictions using the trained polynomial model for Property 10
predictions_poly_prop10 = linear_model_poly_prop10.predict(X_test_poly_prop10)
y_test10 = df_test[target]


# Calculate the Mean Absolute Percentage Error (MAPE) on the test set for Property 10
mape_poly_prop10 = mean_absolute_percentage_error(y_test10, predictions_poly_prop10)

print(f"Mean Absolute Percentage Error (MAPE) on test set with polynomial features for {target}: {mape_poly_prop10:.4f}")



# Define the base directory in Google Drive
home = '/content/drive/MyDrive/Projects/Shell/App/Models'

# Ensure the directory exists
os.makedirs(home, exist_ok=True)

# Define the filename for the saved model including target_10
model_filename_prop10 = os.path.join(home, 'linear_model_poly_target_10.joblib')

# Save the trained model to the file
joblib.dump(linear_model_poly_prop10, model_filename_prop10)

print(f"Model for {target} saved successfully to {model_filename_prop10}")

# Save the poly10 object
poly10_filename = os.path.join(home, 'poly10_features.joblib')
joblib.dump(poly10, poly10_filename)
print(f"PolynomialFeatures object poly10 saved successfully to {poly10_filename}")

Linear Regression model fitted successfully on polynomial features for BlendProperty10.
Mean Absolute Percentage Error (MAPE) on test set with polynomial features for BlendProperty10: 0.0796
Model for BlendProperty10 saved successfully to /content/drive/MyDrive/Projects/Shell/App/Models/linear_model_poly_target_10.joblib
PolynomialFeatures object poly10 saved successfully to /content/drive/MyDrive/Projects/Shell/App/Models/poly10_features.joblib


## Property 5

In [14]:
tn = 5
fn = tn

property_tn = [f'Component{i+1}_Property{fn}' for i in range(5)]
fraction_cols = [f'Component{i+1}_fraction' for i in range(5)]
target_cols = [f"BlendProperty{i}" for i in range(1, 11)]
components = ['Component1', 'Component2', 'Component3', 'Component4', 'Component5']
properties = [f'Property{i}' for i in range(1, 11)]
target = target_cols[tn-1]
targets = target_cols

In [15]:
# prompt: alright lets create property 5, and X5 and y5

property_5 = [f'Component{i+1}_Property5' for i in range(5)]

# Combine selected columns for the new DataFrame
selected_cols_p5 = fraction_cols + property_5

# Create a new DataFrame df3 with only the selected columns
X5 = df_train[selected_cols_p5].copy()
y5 = df_train[targets[4]]
display(X5.head())
display(y5.head())

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property5,Component2_Property5,Component3_Property5,Component4_Property5,Component5_Property5
968,0.07,0.11,0.50,0.17,0.15,-0.105919,0.879857,1.179944,1.718090,-0.654766
240,0.13,0.00,0.48,0.20,0.19,-0.797732,-2.217206,-1.506892,0.503414,-1.550444
819,0.02,0.14,0.32,0.36,0.16,0.842423,0.232389,-0.932049,0.223245,0.173651
692,0.30,0.04,0.00,0.50,0.16,0.083147,0.444720,-0.167392,0.450297,0.836439
420,0.15,0.18,0.00,0.50,0.17,-0.688159,-1.797655,1.203280,0.465399,0.604504


,BlendProperty5
968,0.042194
240,1.072529
819,-0.296572
692,-0.185477
420,-1.358725


In [10]:
# !pip install xgboost
# from xgboost import XGBRegressor # Remove XGBoost import

# Re-initialize and train a TabPFNRegressor model using X5_poly and y5
# Generate polynomial features for X5
poly5 = PolynomialFeatures(degree=2, include_bias=False)
X5_poly = poly5.fit_transform(X5)

# Initialize and train a TabPFNRegressor model
from tabpfn import TabPFNRegressor # Import TabPFNRegressor
model_prop5 = TabPFNRegressor(device='auto',) # Initialize TabPFNRegressor

print("Training TabPFNRegressor model using polynomial features from Property5 and fractions...")
model_prop5.fit(X5_poly, y5)
print("Training complete.")


# Prepare the test data for prediction using df_test
X5_test = df_test[selected_cols_p5].copy()
y5_test = df_test[targets[4]] # Corrected target for property 5
# Transform the test data using the same polynomial features object (poly5)
X5_test_poly = poly5.transform(X5_test)

# Make predictions using the trained TabPFNRegressor model (model_prop5)
predictions_tabpfn_poly5 = model_prop5.predict(X5_test_poly)

# Calculate the Mean Absolute Percentage Error (MAPE) on the test set for the fifth target
mape_tabpfn_poly5 = mean_absolute_percentage_error(y5_test, predictions_tabpfn_poly5)

print(f"Mean Absolute Percentage Error (MAPE) on test set for {targets[4]} with polynomial features using TabPFNRegressor: {mape_tabpfn_poly5:.4f}")

# You can save the TabPFNRegressor model using the specific save function
# from tabpfn.model.loading import save_fitted_tabpfn_model
# import os
# home = '/content/drive/MyDrive/Projects/Shell/App/Models'
# os.makedirs(home, exist_ok=True)
# model_filename_prop5_tabpfn = os.path.join(home, f'tabpfn_model_target_5.tabpfn_fit')
# save_fitted_tabpfn_model(model_prop5, model_filename_prop5_tabpfn)
# print(f"TabPFNRegressor model for {targets[4]} saved successfully to {model_filename_prop5_tabpfn}")

Training TabPFNRegressor model using polynomial features from Property5 and fractions...


tabpfn-v2-regressor.ckpt:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

Training complete.
Mean Absolute Percentage Error (MAPE) on test set for BlendProperty5 with polynomial features using TabPFNRegressor: 0.0723


In [11]:
 # #You can save the TabPFNRegressor model using the specific save function
from tabpfn.model.loading import save_fitted_tabpfn_model
import os
home = '/content/drive/MyDrive/Projects/Shell/App/Models'
os.makedirs(home, exist_ok=True)
model_filename_prop5_tabpfn = os.path.join(home, f'tabpfn_model_target_5.tabpfn_fit')
save_fitted_tabpfn_model(model_prop5, model_filename_prop5_tabpfn)
print(f"TabPFNRegressor model for {targets[4]} saved successfully to {model_filename_prop5_tabpfn}")

TabPFNRegressor model for BlendProperty5 saved successfully to /content/drive/MyDrive/Projects/Shell/App/Models/tabpfn_model_target_5.tabpfn_fit


In [12]:
# Save the poly5 object
import joblib
import os

# Define the base directory in Google Drive
home = '/content/drive/MyDrive/Projects/Shell/App/Models'
os.makedirs(home, exist_ok=True)
poly5_filename = os.path.join(home, 'poly5_features.joblib')

# Assuming poly5 object has been created in a previous cell
# Make sure to run the cell that creates poly5 before running this cell
if 'poly5' in locals():
    joblib.dump(poly5, poly5_filename)
    print(f"PolynomialFeatures object poly5 saved successfully to {poly5_filename}")
else:
    print("Error: poly5 object not found. Please run the cell that creates poly5 first.")

PolynomialFeatures object poly5 saved successfully to /content/drive/MyDrive/Projects/Shell/App/Models/poly5_features.joblib


In [15]:
from tabpfn.model.loading import save_fitted_tabpfn_model, load_fitted_tabpfn_model
import os
import torch

# Define the base directory in Google Drive
home = '/content/drive/MyDrive/Projects/Shell/App/Models'
os.makedirs(home, exist_ok=True)

# Define a temporary local filename
local_model_filename = f'tabpfn_model_target_{tn}_temp.tabpfn_fit'

# Define the final filename for the saved TabPFN model for Property 5 on CPU
google_drive_model_filename = os.path.join(home, f'tabpfn_model_target_{tn}_cpu.tabpfn_fit')

# Save the trained TabPFNRegressor model locally
if 'model_prop5' in locals():
    print(f"Saving TabPFNRegressor model for {targets[4]} locally...")
    save_fitted_tabpfn_model(model_prop5, local_model_filename)
    print(f"Model saved locally to {local_model_filename}")

    # Load the model from the local file, specifying the CPU device
    print(f"Loading model from {local_model_filename} for CPU inference...")
    model_prop5_cpu = load_fitted_tabpfn_model(local_model_filename, device="cpu")
    print("Model loaded successfully for CPU.")

    # Save the CPU-loaded model to Google Drive
    print(f"Saving CPU model to Google Drive at {google_drive_model_filename}...")
    save_fitted_tabpfn_model(model_prop5_cpu, google_drive_model_filename)
    print(f"TabPFNRegressor model for {targets[4]} saved successfully to {google_drive_model_filename}")

    # Optionally, remove the temporary local file
    # os.remove(local_model_filename)
    # print(f"Temporary local model file {local_model_filename} removed.")

else:
    print("Error: model_prop5 object not found. Please run the cell that trains the TabPFNRegressor first.")

Saving TabPFNRegressor model for BlendProperty5 locally...
Model saved locally to tabpfn_model_target_5_temp.tabpfn_fit
Loading model from tabpfn_model_target_5_temp.tabpfn_fit for CPU inference...
Model loaded successfully for CPU.
Saving CPU model to Google Drive at /content/drive/MyDrive/Projects/Shell/App/Models/tabpfn_model_target_5_cpu.tabpfn_fit...
TabPFNRegressor model for BlendProperty5 saved successfully to /content/drive/MyDrive/Projects/Shell/App/Models/tabpfn_model_target_5_cpu.tabpfn_fit


In [17]:
import time
import joblib
import os
from tabpfn.model.loading import load_fitted_tabpfn_model
from sklearn.metrics import mean_absolute_percentage_error

# Define the base directory in Google Drive
home = '/content/drive/MyDrive/Projects/Shell/App/Models'

# Define the filename for the CPU-loaded TabPFN model for Property 5
google_drive_model_filename_cpu = os.path.join(home, f'tabpfn_model_target_{tn}_cpu.tabpfn_fit')

# Define the filename for the polynomial features object for Property 5
poly5_filename = os.path.join(home, 'poly5_features.joblib')

# Load the CPU-loaded TabPFN model
print(f"Loading CPU model from {google_drive_model_filename_cpu}...")
try:
    model_prop5_cpu_loaded = load_fitted_tabpfn_model(google_drive_model_filename_cpu, device="cpu")
    print("CPU model loaded successfully.")
except FileNotFoundError:
    print(f"Error: Model file not found at {google_drive_model_filename_cpu}. Please ensure the previous save cell was run.")
    model_prop5_cpu_loaded = None # Set to None to prevent further execution

# Load the polynomial features object
print(f"Loading PolynomialFeatures object from {poly5_filename}...")
try:
    poly5_loaded = joblib.load(poly5_filename)
    print("PolynomialFeatures object loaded successfully.")
except FileNotFoundError:
    print(f"Error: PolynomialFeatures file not found at {poly5_filename}. Please ensure the previous save cell was run.")
    poly5_loaded = None # Set to None to prevent further execution


if model_prop5_cpu_loaded is not None and poly5_loaded is not None:
    # Prepare the test data (assuming df_test and selected_cols_p5 are available from previous cells)
    # If not, you would need to load df_test and define selected_cols_p5 here
    if 'df_test' in locals() and 'selected_cols_p5' in locals() and 'targets' in locals():
        # Use only the first 5 test samples
        X5_test_subset = df_test[selected_cols_p5].head(1).copy()
        y5_test_subset = df_test[targets[4]].head(1) # Corrected target for property 5

        # Transform the test data subset using the loaded polynomial features object
        print("Transforming the first 5 test samples with loaded PolynomialFeatures object...")
        X5_test_poly_loaded_subset = poly5_loaded.transform(X5_test_subset)
        print("Test data subset transformation complete.")

        # Measure prediction time
        print("Making predictions on the first 5 test samples and measuring time...")
        start_time = time.time()
        predictions_tabpfn_poly5_loaded_subset = model_prop5_cpu_loaded.predict(X5_test_poly_loaded_subset)
        end_time = time.time()
        prediction_time = end_time - start_time
        print(f"Prediction complete for 5 samples. Time taken: {prediction_time:.4f} seconds.")


        # Calculate the Mean Absolute Percentage Error (MAPE) on the test set subset
        mape_tabpfn_poly5_loaded_subset = mean_absolute_percentage_error(y5_test_subset, predictions_tabpfn_poly5_loaded_subset)

        print(f"\nEvaluation of CPU-loaded TabPFNRegressor on the first 5 test samples for {targets[4]}:")
        print(f"  Mean Absolute Percentage Error (MAPE): {mape_tabpfn_poly5_loaded_subset:.4f}")

    else:
        print("Error: df_test, selected_cols_p5, or targets not found. Please ensure the data loading and preparation cells were run.")

Loading CPU model from /content/drive/MyDrive/Projects/Shell/App/Models/tabpfn_model_target_5_cpu.tabpfn_fit...
CPU model loaded successfully.
Loading PolynomialFeatures object from /content/drive/MyDrive/Projects/Shell/App/Models/poly5_features.joblib...
PolynomialFeatures object loaded successfully.
Transforming the first 5 test samples with loaded PolynomialFeatures object...
Test data subset transformation complete.
Making predictions on the first 5 test samples and measuring time...


KeyboardInterrupt: 

In [ ]:
import time
import joblib
import os
from tabpfn.model.loading import load_fitted_tabpfn_model
from sklearn.metrics import mean_absolute_percentage_error

# Define the base directory in Google Drive
home = '/content/drive/MyDrive/Projects/Shell/App/Models'

# Define the filename for the CPU-loaded TabPFN model for Property 5
google_drive_model_filename_cpu = os.path.join(home, f'tabpfn_model_target_{tn}_cpu.tabpfn_fit')

# Define the filename for the polynomial features object for Property 5
poly5_filename = os.path.join(home, 'poly5_features.joblib')

# Load the CPU-loaded TabPFN model
print(f"Loading CPU model from {google_drive_model_filename_cpu}...")
try:
    model_prop5_cpu_loaded = load_fitted_tabpfn_model(google_drive_model_filename_cpu, device="cpu")
    print("CPU model loaded successfully.")
except FileNotFoundError:
    print(f"Error: Model file not found at {google_drive_model_filename_cpu}. Please ensure the previous save cell was run.")
    model_prop5_cpu_loaded = None # Set to None to prevent further execution

# Load the polynomial features object
print(f"Loading PolynomialFeatures object from {poly5_filename}...")
try:
    poly5_loaded = joblib.load(poly5_filename)
    print("PolynomialFeatures object loaded successfully.")
except FileNotFoundError:
    print(f"Error: PolynomialFeatures file not found at {poly5_filename}. Please ensure the previous save cell was run.")
    poly5_loaded = None # Set to None to prevent further execution


if model_prop5_cpu_loaded is not None and poly5_loaded is not None:
    # Prepare the test data (assuming df_test and selected_cols_p5 are available from previous cells)
    # If not, you would need to load df_test and define selected_cols_p5 here
    if 'df_test' in locals() and 'selected_cols_p5' in locals() and 'targets' in locals():
        # Use only the first test sample
        X5_test_subset = df_test[selected_cols_p5].head(1).copy()
        y5_test_subset = df_test[targets[4]].head(1) # Corrected target for property 5

        # Transform the test data subset using the loaded polynomial features object
        print("Transforming the first test sample with loaded PolynomialFeatures object...")
        X5_test_poly_loaded_subset = poly5_loaded.transform(X5_test_subset)
        print("Test data subset transformation complete.")

        # Measure prediction time
        print("Making prediction on the first test sample and measuring time...")
        start_time = time.time()
        predictions_tabpfn_poly5_loaded_subset = model_prop5_cpu_loaded.predict(X5_test_poly_loaded_subset)
        end_time = time.time()
        prediction_time = end_time - start_time
        print(f"Prediction complete for 1 sample. Time taken: {prediction_time:.4f} seconds.")

        # Display the prediction
        print("\nPrediction for the first test sample:")
        display(predictions_tabpfn_poly5_loaded_subset)


        # Calculate the Mean Absolute Percentage Error (MAPE) on the test set subset
        mape_tabpfn_poly5_loaded_subset = mean_absolute_percentage_error(y5_test_subset, predictions_tabpfn_poly5_loaded_subset)

        print(f"\nEvaluation of CPU-loaded TabPFNRegressor on the first test sample for {targets[4]}:")
        print(f"  Mean Absolute Percentage Error (MAPE): {mape_tabpfn_poly5_loaded_subset:.4f}")

    else:
        print("Error: df_test, selected_cols_p5, or targets not found. Please ensure the data loading and preparation cells were run.")

Loading CPU model from /content/drive/MyDrive/Projects/Shell/App/Models/tabpfn_model_target_5_cpu.tabpfn_fit...
CPU model loaded successfully.
Loading PolynomialFeatures object from /content/drive/MyDrive/Projects/Shell/App/Models/poly5_features.joblib...
PolynomialFeatures object loaded successfully.
Transforming the first test sample with loaded PolynomialFeatures object...
Test data subset transformation complete.
Making prediction on the first test sample and measuring time...


## Property 7

In [ ]:
from itertools import combinations
from scipy.special import comb

def generate_mixture_features(data):
    """
    Generate symmetric and weighted nonlinear interactions between fuel weights and properties.
    The input 'data' should contain weights in the first 5 columns/elements and properties in the next 5.

    :param data: np.ndarray, pd.DataFrame, or list of shape (n_samples, 10) or (10,)
    :return: pd.DataFrame with generated features.
    """
    # Convert input to numpy array and handle single row/list input
    if isinstance(data, pd.DataFrame):
        data_array = data.values
    elif isinstance(data, list):
        data_array = np.array(data)
    elif isinstance(data, np.ndarray):
        data_array = data
    else:
        raise TypeError("Input data must be a pandas DataFrame, numpy array, or list.")

    # Reshape single row/list input to 2D array
    if data_array.ndim == 1:
        data_array = data_array.reshape(1, -1)

    # Ensure the input has 10 columns (5 weights + 5 properties)
    if data_array.shape[1] != 10:
        raise ValueError("Input data must have 10 columns/elements (5 weights and 5 properties).")

    # Separate weights and properties
    W = data_array[:, :5]
    P = data_array[:, 5:]


    n_samples, n_fuels = W.shape
    features = {}

    # Original weights and properties
    for i in range(n_fuels):
        features[f'w{i+1}'] = W[:, i]
        features[f'p{i+1}'] = P[:, i]
        features[f'w{i+1}_p{i+1}'] = W[:, i] * P[:, i]  # weighted property

    # --- 1. Weighted sum of properties ---
    features['weighted_sum'] = np.sum(W * P, axis=1)

    # --- 2. Weighted square of properties ---
    features['weighted_sum_sq'] = np.sum(W * P**2, axis=1)

    # --- 3. Weighted tanh of properties ---
    features['weighted_tanh'] = np.sum(W * np.tanh(P), axis=1)

    # --- 4. Weighted exponential ---
    # features['weighted_exp'] = np.sum(W * np.exp(P), axis=1)
    # Clip P before exponential to avoid overflow
    safe_exp = np.exp(np.clip(P, a_min=None, a_max=50))  # 50 is safe upper bound
    features['weighted_exp'] = np.sum(W * safe_exp, axis=1)


    # --- 5. Weighted logarithm (clip to avoid -inf) ---
    # features['weighted_log'] = np.sum(W * np.log(np.clip(P, 1e-6, None)), axis=1)
    features['weighted_log'] = np.sum(W * np.log(np.clip(P, 1e-6, None)), axis=1)


    # --- 6. Pairwise interactions (symmetric, weighted) ---
    for i, j in combinations(range(n_fuels), 2):
        pij = P[:, i] * P[:, j]
        wij = W[:, i] * W[:, j]
        features[f'pair_p{i+1}p{j+1}'] = pij
        features[f'weighted_pair_p{i+1}p{j+1}'] = pij * wij

    # --- 7. Triple interactions (weighted & symmetric) ---
    for i, j, k in combinations(range(n_fuels), 3):
        pij = P[:, i] * P[:, j] * P[:, k]
        wij = W[:, i] * W[:, j] * W[:, k]
        features[f'triplet_p{i+1}{j+1}{k+1}'] = pij
        features[f'weighted_triplet_p{i+1}{j+1}{k+1}'] = pij * wij

    # --- 8. Power series + weight modulated ---
    for power in [2, 3, 4]:
        features[f'power_sum_{power}'] = np.sum(W * P**power, axis=1)

    # --- 9. Log-weighted property (prevent log(0)) ---
    logW = np.log(np.clip(W, 1e-6, None))
    features['log_weighted_p'] = np.sum(logW * P, axis=1)

    # --- 10. Symmetric polynomial combinations (elementary symmetric) ---
    # Up to degree 5 (since you have 5 fuels)
    for r in range(1, 6):
        key = f'e_sym_poly_r{r}'
        val = np.zeros(n_samples)
        for idx in combinations(range(n_fuels), r):
            prod_p = np.prod(P[:, idx], axis=1)
            val += prod_p
        features[key] = val

    # --- 11. Weighted interaction difference (symmetry in differences) ---
    for i, j in combinations(range(n_fuels), 2):
        diff = P[:, i] - P[:, j]
        wdiff = W[:, i] * W[:, j]
        features[f'weighted_diff_p{i+1}{j+1}'] = diff * wdiff

    # --- 12. Mean, max, min (weighted) ---
    total_weight = np.sum(W, axis=1, keepdims=True)
    weighted_mean = np.sum(W * P, axis=1) / np.clip(total_weight.squeeze(), 1e-6, None)
    features['weighted_mean'] = weighted_mean
    features['max_prop'] = np.max(P, axis=1)
    features['min_prop'] = np.min(P, axis=1)

    # --- 13. Weighted cross-log terms ---
    for i, j in combinations(range(n_fuels), 2):
        log_mix = np.log(np.clip(P[:, i] + P[:, j], 1e-6, None))
        wij = W[:, i] * W[:, j]
        features[f'logsum_p{i+1}{j+1}'] = log_mix * wij

    # --- 14. Inverse + weighted inverse ---
    # features['inv_prop_sum'] = np.sum(W / np.clip(P, 1e-6, None), axis=1)
    features['inv_prop_sum'] = np.sum(W / np.clip(P, 1e-6, None), axis=1)


    # --- 15. Weighted relu (max(p, 0)) ---
    relu = np.maximum(P, 0)
    features['weighted_relu'] = np.sum(W * relu, axis=1)

    # --- 16. Weighted sin/cos transforms ---
    features['weighted_sin'] = np.sum(W * np.sin(P), axis=1)
    features['weighted_cos'] = np.sum(W * np.cos(P), axis=1)

    # --- 17. Normalized properties ---
    prop_sum = np.sum(P, axis=1, keepdims=True)
    normalized_P = P / np.clip(prop_sum, 1e-6, None)
    for i in range(n_fuels):
        features[f'norm_p{i+1}'] = normalized_P[:, i]

    # --- 18. Product of all p's and all w's ---
    features['total_product_p'] = np.prod(P, axis=1)
    features['total_product_w'] = np.prod(W, axis=1)

    # --- 19. Mixed entropic form ---
    # entropy_like = -np.sum(W * np.log(np.clip(W, 1e-6, None)), axis=1)
    # features['entropy_weights'] = entropy_like

    # Convert to DataFrame
    df = pd.DataFrame(features)

    return df

In [ ]:
# Set up for Property 7
tn = 7
fn = tn

property_tn = [f'Component{i+1}_Property{fn}' for i in range(5)]
fraction_cols = [f'Component{i+1}_fraction' for i in range(5)]
target_cols = [f"BlendProperty{i}" for i in range(1, 11)]
components = ['Component1', 'Component2', 'Component3', 'Component4', 'Component5']
properties = [f'Property{i}' for i in range(1, 11)]
target = target_cols[tn-1]

# Select relevant columns for Property 7 from training data
train_data_prop7 = df_train[fraction_cols + property_tn]
y_train_prop7 = df_train[target]

# Generate mixture features for the training data using the modified function
X_train_prop7_mixture = generate_mixture_features(train_data_prop7)

# Identify columns to concatenate from df_train (excluding target columns and original fraction/property columns for property 7)
cols_to_concat = [col for col in df_train.columns if col not in target_cols + fraction_cols + property_tn]

# Select these "other features" columns from df_train
df_train_other_features = df_train[cols_to_concat]

# Reset index of mixture features and other features to ensure proper concatenation
X_train_prop7_mixture = X_train_prop7_mixture.reset_index(drop=True)
df_train_other_features = df_train_other_features.reset_index(drop=True)

# Concatenate mixture features with the other features for training
X_train_prop7 = pd.concat([X_train_prop7_mixture, df_train_other_features], axis=1)

# Display the head of the concatenated training DataFrame to verify
print("Head of concatenated training features for Property 7:")
display(X_train_prop7.head())


# Select relevant columns for Property 7 from test data
test_data_prop7 = df_test[fraction_cols + property_tn]
y_test_prop7 = df_test[target]

# Generate mixture features for the test data using the modified function
X_test_prop7_mixture = generate_mixture_features(test_data_prop7)

# Select the same "other features" columns from the test DataFrame
df_test_other_features = df_test[cols_to_concat] # Use the same cols_to_concat identified from df_train

# Reset index of mixture features and other features to ensure proper concatenation
X_test_prop7_mixture = X_test_prop7_mixture.reset_index(drop=True)
df_test_other_features = df_test_other_features.reset_index(drop=True)

# Concatenate mixture features with the other features for testing
X_test_prop7 = pd.concat([X_test_prop7_mixture, df_test_other_features], axis=1)

# Display the head of the concatenated test DataFrame to verify
print("\nHead of concatenated test features for Property 7:")
display(X_test_prop7.head())


# Initialize the TabPFNRegressor model
# Using default parameters for simplicity
model_prop7 = TabPFNRegressor(device='auto',) # Use 'cuda' if a GPU is available


# Fit the model to the training data
print(f"\nTraining TabPFNRegressor for {target} with mixture and other features...")
model_prop7.fit(X_train_prop7, y_train_prop7)
print(f"Training complete for {target}.")

# Make predictions on the training data
y_train_pred_prop7 = model_prop7.predict(X_train_prop7)

# Calculate the Mean Squared Error (MSE) on the training set
mse_train_prop7 = mean_squared_error(y_train_prop7, y_train_pred_prop7)
print(f"Mean Squared Error (MSE) on the training set for {target}: {mse_train_prop7}")


# Make predictions on the test data
y_test_pred_prop7 = model_prop7.predict(X_test_prop7)


# Calculate the Mean Squared Error (MSE) on the test set
mse_test_prop7 = mean_squared_error(y_test_prop7, y_test_pred_prop7)
print(f"Mean Squared Error (MSE) on the test set for {target}: {mse_test_prop7}")

# Calculate the Mean Absolute Percentage Error (MAPE) on the test set
mape_test_prop7 = mean_absolute_percentage_error(y_test_prop7, y_test_pred_prop7)
print(f"Mean Absolute Percentage Error (MAPE) on the test set for {target}: {mape_test_prop7}")



Head of concatenated training features for Property 7:


,w1,p1,w1_p1,w2,p2,w2_p2,w3,p3,w3_p3,w4,...,Component1_Property9,Component2_Property9,Component3_Property9,Component4_Property9,Component5_Property9,Component1_Property10,Component2_Property10,Component3_Property10,Component4_Property10,Component5_Property10
0,0.07,0.898442,0.062891,0.11,-1.151777,-0.126695,0.50,-1.025072,-0.512536,0.17,...,0.043551,-0.449533,-0.715677,-0.273495,-0.879258,0.593625,-0.187536,-0.465085,-1.268823,0.698367
1,0.13,-1.219600,-0.158548,0.00,0.929881,0.000000,0.48,-1.432883,-0.687784,0.20,...,-0.566695,-0.176404,-0.762321,1.369383,0.095009,-1.087136,1.286728,-0.499292,1.853660,-0.911848
2,0.02,-0.966331,-0.019327,0.14,-0.899544,-0.125936,0.32,-1.503322,-0.481063,0.36,...,-0.816097,-0.064482,1.303794,1.286489,0.409193,-0.966469,-1.805664,-2.515751,0.752693,-0.955162
3,0.30,-0.714839,-0.214452,0.04,-0.790425,-0.031617,0.00,-1.420965,-0.000000,0.50,...,0.213288,-0.395155,1.326028,-1.285228,-1.934297,-0.671642,0.738591,-0.232804,-0.472313,-0.944420
4,0.15,-0.965264,-0.144790,0.18,1.137836,0.204811,0.00,0.210736,0.000000,0.50,...,0.296198,2.179884,0.738021,0.506321,-1.472864,-1.419736,-0.505352,0.158943,1.029472,0.154698



Head of concatenated test features for Property 7:


,w1,p1,w1_p1,w2,p2,w2_p2,w3,p3,w3_p3,w4,...,Component1_Property9,Component2_Property9,Component3_Property9,Component4_Property9,Component5_Property9,Component1_Property10,Component2_Property10,Component3_Property10,Component4_Property10,Component5_Property10
0,0.00,-0.871244,-0.000000,0.00,0.681215,0.000000,0.45,-0.550337,-0.247652,0.50,...,-0.533128,-0.390062,0.011976,0.162902,-0.565074,0.947255,-0.943861,0.662869,0.608152,-0.832499
1,0.00,-0.974684,-0.000000,0.19,-0.368218,-0.069961,0.32,0.608921,0.194855,0.49,...,1.083342,-1.380843,0.584183,-0.247232,-1.228516,-1.022617,0.095697,-0.770686,-1.518437,-0.202550
2,0.17,1.136070,0.193132,0.12,-1.266772,-0.152013,0.15,2.622763,0.393414,0.44,...,-2.419364,1.854442,-0.500816,0.105098,0.769856,-1.373593,-1.409000,-0.831177,-0.277030,0.913894
3,0.16,1.347039,0.215526,0.31,0.942681,0.292231,0.16,-1.247695,-0.199631,0.13,...,-0.502806,-0.704353,-0.695431,-0.371984,0.526367,0.802291,-1.200120,0.219434,0.735266,0.217070
4,0.21,0.164761,0.034600,0.23,-1.119671,-0.257524,0.06,0.745061,0.044704,0.50,...,0.016362,0.980677,-2.033135,-0.511391,-0.050990,0.496030,0.606830,0.522016,-0.138127,-0.275316



Training TabPFNRegressor for BlendProperty7 with mixture and other features...
Training complete for BlendProperty7.
Mean Squared Error (MSE) on the training set for BlendProperty7: 0.0010857877458235843
Mean Squared Error (MSE) on the test set for BlendProperty7: 0.010203085298786677
Mean Absolute Percentage Error (MAPE) on the test set for BlendProperty7: 0.4502231661422518


In [ ]:
# You can save the model if needed, similar to the Linear Regression models
# import joblib
# import os
# home = '/content/drive/MyDrive/Projects/Shell/App/Models'
# os.makedirs(home, exist_ok=True)
# model_filename_prop7 = os.path.join(home, f'tabpfn_model_target_{tn}.joblib')
# joblib.dump(model_prop7, model_filename_prop7)
# print(f"Model for {target} saved successfully to {model_filename_prop7}")

In [ ]:
from tabpfn.model.loading import save_fitted_tabpfn_model
import os

# Define the base directory in Google Drive
home = '/content/drive/MyDrive/Projects/Shell/App/Models'
os.makedirs(home, exist_ok=True)

# Define the filename for the saved TabPFN model for Property 7
model_filename_prop7_tabpfn = os.path.join(home, f'tabpfn_model_target_{tn}.tabpfn_fit')

# Save the trained TabPFNRegressor model
# Make sure the model_prop7 object exists from a previous cell execution
if 'model_prop7' in locals():
    save_fitted_tabpfn_model(model_prop7, model_filename_prop7_tabpfn)
    print(f"TabPFNRegressor model for {target} saved successfully to {model_filename_prop7_tabpfn}")
else:
    print("Error: model_prop7 object not found. Please run the cell that trains the TabPFNRegressor first.")

TabPFNRegressor model for BlendProperty7 saved successfully to /content/drive/MyDrive/Projects/Shell/App/Models/tabpfn_model_target_7.tabpfn_fit


## Property 8

In [ ]:
# ## piecewise update code for 3
# df_full = pd.read_csv('validation_predictions_all_folds.csv')
# # lready have a DataFrame 'df' with the full column list:
# tn = 3

# property_number = tn  # change this to the property you're interested in

# # Pattern to search
# property_str = f"Property{property_number}"

# # Filter columns that contain the desired property
# matching_columns = [col for col in df_full.columns if property_str in col]

# # Display or extract those columns
# df_property = df_full[matching_columns]

# df_property.reset_index(drop=False, inplace=True)

# df_property.rename(columns={'index': 'ID'}, inplace=True)
# display(df_property.head())

# df_u_train, df_u_test = train_test_split(df_property, test_size=0.2, random_state=42)


# threshold = 0.2
# df_a = df_u_train[(df_u_train['BlendProperty3_pred'].abs()< threshold) | (df_u_train['BlendProperty3_true'].abs()< threshold)].copy()
# df_b = df_u_train[(df_u_train['BlendProperty3_pred'].abs()>= threshold) & (df_u_train['BlendProperty3_true'].abs()>= threshold)].copy()

# def piecewise_model(x, params, boundaries=np.linspace(-0.4, 0.4, 11)[1:-1]):
#     """
#     x: a single float value
#     params: list of 20 parameters [A1, B1, A2, B2, ..., A10, B10]
#     boundaries: 9 values that divide x into 10 regions
#     """
#     # Unpack parameters
#     segments = [(params[i], params[i+1]) for i in range(0, 20, 2)]

#     # Piecewise logic using boundaries
#     for i, bound in enumerate(boundaries):
#         if x < bound:
#             A, B = segments[i]
#             return A * x + B
#     # If x is greater than all boundaries, use the last segment
#     A, B = segments[-1]
#     return A * x + B


# # Initial guess for 10 parameters

# def mape(y_true, y_pred):
#     return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100

# def cost_function(params, x, y_true):
#     func = lambda t: piecewise_model(t, params)
#     y_pred = x.apply(func)
#     return mape(y_true, y_pred)


# from scipy.optimize import minimize

# # Example synthetic data
# np.random.seed(0)
# y_first_pred =  df_a['BlendProperty3_pred']   #np.linspace(0, 1, 200)
# y3_data =    df_a['BlendProperty3_true']



# # Run optimizer
# result = minimize(cost_function, init_params, args=(y_first_pred, y3_data), method='L-BFGS-B')
# result = minimize(cost_function, result.x, args=(y_first_pred, y3_data), method='L-BFGS-B')
# # Optimized parameters
# opt_params = result.x

# print("Optimized Parameters:", opt_params)


In [ ]:
# piecewise update code for target 8

df_full = pd.read_csv('validation_predictions_all_folds.csv')

tn = 8
property_number = tn

# Pattern to search for Property 8 columns
property_str = f"Property{property_number}"

# Filter columns that contain the desired property
# Assuming the prediction and true columns follow a pattern like 'BlendPropertyX_pred' and 'BlendPropertyX_true'
pred_col = f"BlendProperty{property_number}_pred"
true_col = f"BlendProperty{property_number}_true"

# Select only the relevant columns for Property 8 from df_full
if pred_col in df_full.columns and true_col in df_full.columns:
    df_property_8 = df_full[[pred_col, true_col]].copy()
else:
    raise ValueError(f"Columns '{pred_col}' or '{true_col}' not found in df_full.")


df_property_8.reset_index(drop=False, inplace=True)
df_property_8.rename(columns={'index': 'ID'}, inplace=True)



df_u_train_8, df_u_test_8 = train_test_split(df_property_8, test_size=0.2, random_state=42)

# Define threshold (you might need to adjust this based on the distribution of Property 8)
threshold_8 = 0.2 # Example threshold, adjust as needed

# Split data based on threshold for target 8
df_a_8 = df_u_train_8[(df_u_train_8[pred_col].abs() < threshold_8) | (df_u_train_8[true_col].abs() < threshold_8)].copy()
df_b_8 = df_u_train_8[(df_u_train_8[pred_col].abs() >= threshold_8) & (df_u_train_8[true_col].abs() >= threshold_8)].copy()

n=10
# Define MAPE cost function
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100

def piecewise_model(x, params, boundaries=np.linspace(-threshold, threshold, n+1)[1:-1]):
    """
    x: a single float value
    params: list of 20 parameters [A1, B1, A2, B2, ..., A10, B10]
    boundaries: 9 values that divide x into 10 regions
    """
    # Unpack parameters
    segments = [(params[i], params[i+1]) for i in range(0, 20, 2)]

    # Piecewise logic using boundaries
    for i, bound in enumerate(boundaries):
        if x < bound:
            A, B = segments[i]
            return A * x + B
    # If x is greater than all boundaries, use the last segment
    A, B = segments[-1]
    return A * x + B

# Define the cost function for optimization using the original piecewise_model
def cost_function(params, x, y_true):
    # Reuse the original piecewise_model function
    func = lambda t: piecewise_model(t, params, )
    y_pred = x.apply(func)
    return mape(y_true, y_pred)



init_params_8 = np.zeros(2*n) # Use the same initial guess as for Property 3


# Example synthetic data (using data from df_a_8 for optimization)
y_pred_a_8 =  df_a_8[pred_col]
y_true_a_8 =  df_a_8[true_col]

from scipy.optimize import minimize

# Initial guess for 10 parameters
init_params_8 = np.random.uniform(-1, 1, 20)
# Run optimizer
result_8 = minimize(cost_function, init_params_8, args=(y_pred_a_8, y_true_a_8, ), method='L-BFGS-B')
result_8 = minimize(cost_function, result_8.x, args=(y_pred_a_8, y_true_a_8, ), method='L-BFGS-B')

# Optimized parameters
opt_params_8 = result_8.x

print(f"Optimized Parameters for Property {property_number}:", opt_params_8)

# Now you can use the optimized parameters and boundaries to define the final piecewise function for Property 8
# func_8 = lambda t: piecewise_model(t, opt_params_8, boundaries_8) # Use the original piecewise_model

# You might also want to evaluate the performance on df_b_8 or the full test set df_u_test_8
# using the optimized function.

Optimized Parameters for Property 8: [ 1.05132131  0.14051208 -0.19599418 -0.12772231 -0.45243696 -0.05234876
  0.90743032  0.04327284  0.245341    0.01030205 -0.45654816  0.00119642
  0.2647378  -0.01674035 -0.22671536  0.04704649  0.88503473 -0.11061753
 -0.09296722  0.07304861]


In [ ]:
ndf = df_property_8.copy()

In [ ]:
import matplotlib.pyplot as plt

func8 = lambda t: piecewise_model(t, opt_params_8)

ndf = df_property_8.copy()

ndf.loc[ndf[pred_col].abs() < threshold_8, pred_col] = ndf[ndf[pred_col].abs() < threshold_8][pred_col].apply(func8)





In [ ]:
# Assuming ndf and df_property_8 DataFrames exist from previous cells
# Assuming pred_col and true_col variables are defined

# Evaluate the piecewise corrected predictions (ndf)
mape_corrected = mean_absolute_percentage_error(ndf[true_col], ndf[pred_col])
mse_corrected = mean_squared_error(ndf[true_col], ndf[pred_col])

print(f"Evaluation of Piecewise Corrected Predictions for {true_col}:")
print(f"  Mean Absolute Percentage Error (MAPE): {mape_corrected:.4f}")
print(f"  Mean Squared Error (MSE): {mse_corrected:.4f}")

# print()
print("_" * 75) # Separator for clarity
print()

# Evaluate the initial predictions (df_property_8)
mape_initial = mean_absolute_percentage_error(df_property_8[true_col], df_property_8[pred_col])
mse_initial = mean_squared_error(df_property_8[true_col], df_property_8[pred_col])

print(f"Evaluation of Initial Predictions for {true_col}:")
print(f"  Mean Absolute Percentage Error (MAPE): {mape_initial:.4f}")
print(f"  Mean Squared Error (MSE): {mse_initial:.4f}")

Evaluation of Piecewise Corrected Predictions for BlendProperty8_true:
  Mean Absolute Percentage Error (MAPE): 0.2891
  Mean Squared Error (MSE): 0.0110
___________________________________________________________________________

Evaluation of Initial Predictions for BlendProperty8_true:
  Mean Absolute Percentage Error (MAPE): 0.6144
  Mean Squared Error (MSE): 0.0098


In [ ]:
import pickle
import os

# Define the base directory in Google Drive
home = '/content/drive/MyDrive/Projects/Shell/App/Models'

# Ensure the directory exists
os.makedirs(home, exist_ok=True)

# Define the filename for the saved function
function_filename = os.path.join(home, 'piecewise_function_prop8.pkl') # Adding prop8 to distinguish

# Save the function to a file
with open(function_filename, 'wb') as f:
    pickle.dump(func8, f)

print(f"Function saved to {function_filename}")

PicklingError: Can't pickle <function <lambda> at 0x7d49d9ffaca0>: attribute lookup <lambda> on __main__ failed

In [ ]:


def augment_data(df, num_augmentations_per_row, property_num, shuffle=True):
    """
    Augments the DataFrame by permuting componentA fraction and property columns within each row.

    Args:
        df: The input pandas DataFrame containing component fractions, properties, and a target.
        num_augmentations_per_row: The number of augmented rows to generate per original row.
        property_num: The number of the property to use for augmentation (e.g., 1 for Property1).
        shuffle: Whether to shuffle the final augmented DataFrame (default is True).

    Returns:
        A new DataFrame with augmented data.
    """
    # Identify relevant columns
    fraction_cols = [f'Component{i+1}_fraction' for i in range(5)]
    property_cols = [f'Component{i+1}_Property{property_num}' for i in range(5)]
    target_col = f"BlendProperty{property_num}" # Assuming target column follows this naming convention


    # Create a list of tuples, where each tuple contains the fraction and property column for a component
    component_col_pairs = [(fraction_cols[i], property_cols[i]) for i in range(5)]

    # Create an empty list to store the augmented rows (as dictionaries for easy creation of DataFrame)
    augmented_rows_list = []

    # Iterate through each row of the original dataframe
    for index, row in df.iterrows():
        original_row_data = row.to_dict()
        augmented_rows_list.append(original_row_data) # Add the original row

        # Generate a few random permutations of the component pairs
        # Ensure unique permutations are sampled if num_augmentations_per_row is large
        all_permutations = list(itertools.permutations(component_col_pairs))
        num_samples = min(num_augmentations_per_row, len(all_permutations))
        permutations_for_row = random.sample(all_permutations, num_samples)


        for perm in permutations_for_row:
            new_row_data = {}
            # Assign the target value
            new_row_data[target_col] = original_row_data[target_col]

            # Assign the permuted fraction and property values while keeping original column names
            for i in range(5):
                original_fraction_col = fraction_cols[i]
                original_property_col = property_cols[i]
                permuted_fraction_col_name = perm[i][0]
                permuted_property_col_name = perm[i][1]

                new_row_data[original_fraction_col] = original_row_data[permuted_fraction_col_name]
                new_row_data[original_property_col] = original_row_data[permuted_property_col_name]

            augmented_rows_list.append(new_row_data)

    # Create a new dataframe from the list of augmented rows
    df_augmented = pd.DataFrame(augmented_rows_list)

    # Ensure columns are in the original order
    df_augmented = df_augmented[fraction_cols + property_cols + [target_col]]

    if shuffle:
        df_augmented = df_augmented.sample(frac=1, random_state=42).reset_index(drop=True)


    return df_augmented

In [ ]:
# # Apply the augmentation function to the training data
# # We'll create 3 augmented rows per original row for Property 1 and shuffle the result
# df_set1 = df_train[fraction_cols + property_tn  + [target]]

# augmented_df_set1 = augment_data(df_set1, num_augmentations_per_row=3, property_num=1, shuffle=True)

# # Separate features (X) and target (y)
# X_train_1_augmented = augmented_df_set1.drop(columns=[target])
# y_train_1_augmented = augmented_df_set1[target]

# display(X_train_1_augmented.head())
# display(y_train_1_augmented.head())

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1
0,0.12,0.32,0.01,0.37,0.18,-0.971985,-1.718895,-0.220068,0.425271,-0.977027
1,0.28,0.23,0.16,0.14,0.19,0.231383,0.497659,-0.872661,0.207838,-0.329878
2,0.00,0.00,0.50,0.00,0.50,-1.455473,0.206283,-0.037829,-0.741048,0.200032
3,0.00,0.36,0.11,0.50,0.03,-0.311093,1.061484,1.032029,-0.005169,-0.010389
4,0.00,0.49,0.00,0.35,0.16,-1.380062,1.454508,0.570584,2.149234,1.032029


,BlendProperty1
0,-0.124281
1,0.948078
2,-0.750112
3,1.165675
4,1.050554


In [ ]:
# Example usage of the modified generate_mixture_features function

# 1. With a single row of DataFrame data
# Assuming df_train is your training DataFrame
if not df_train.empty:
    sample_row = df_train.iloc[0] # Get the first row
    # Extract weights and properties for the specific property number (e.g., Property1, fn=1)
    # You would need to define which property to use based on your task
    # For demonstration, let's assume fn is defined and select columns accordingly
    current_property_fn = 1 # Replace with the desired property number
    fraction_cols_sample = [f'Component{i+1}_fraction' for i in range(5)]
    property_cols_sample = [f'Component{i+1}_Property{current_property_fn}' for i in range(5)]
    sample_data = sample_row[fraction_cols_sample + property_cols_sample]

    features_from_row = generate_mixture_features(sample_data)
    print("\nFeatures from a single DataFrame row:")
    display(features_from_row)
else:
    print("df_train is empty. Cannot demonstrate with a DataFrame row.")


# 2. With a list
sample_list = [0.1, 0.2, 0.3, 0.2, 0.2, 1.0, 2.0, 3.0, 4.0, 5.0] # 5 weights, 5 properties
features_from_list = generate_mixture_features(sample_list)
print("\nFeatures from a list:")
display(features_from_list)

# 3. With a numpy array
sample_array = np.array([[0.1, 0.2, 0.3, 0.2, 0.2, 1.0, 2.0, 3.0, 4.0, 5.0],
                         [0.5, 0.1, 0.1, 0.1, 0.2, 5.0, 4.0, 3.0, 2.0, 1.0]]) # 2 samples
features_from_array = generate_mixture_features(sample_array)
print("\nFeatures from a numpy array:")
display(features_from_array)

# 4. With a DataFrame
# Assuming df_train has the required columns, select the relevant ones for a property
if not df_train.empty:
    # For demonstration, using Property 1 columns (assuming fn=1)
    current_property_fn = 1
    fraction_cols_df = [f'Component{i+1}_fraction' for i in range(5)]
    property_cols_df = [f'Component{i+1}_Property{current_property_fn}' for i in range(5)]
    sample_df = df_train[fraction_cols_df + property_cols_df].head() # Use a few rows

    features_from_df = generate_mixture_features(sample_df)
    print("\nFeatures from a DataFrame:")
    display(features_from_df)
else:
     print("df_train is empty. Cannot demonstrate with a DataFrame.")

In [ ]:
import numpy as np

def piecewise_model(x, params, boundaries=np.linspace(-0.1, 0.1, 6)[1:-1]):
    """
    x: a single float value
    params: list of 10 parameters [A1, B1, A2, B2, ..., A5, B5]
    boundaries: 4 values that divide x into 5 regions
    """
    # Unpack parameters (10 parameters for 5 segments)
    segments = [(params[i], params[i+1]) for i in range(0, 10, 2)]

    # Piecewise logic using boundaries
    for i, bound in enumerate(boundaries):
        if x < bound:
            A, B = segments[i]
            return A * x + B
    # If x is greater than all boundaries, use the last segment
    A, B = segments[-1]
    return A * x + B

In [ ]:
import joblib
import os

# Define the base directory in Google Drive
home = '/content/drive/MyDrive/Projects/Shell/App/Models'

# Ensure the directory exists
os.makedirs(home, exist_ok=True)

# Define the filenames for the saved parameters and threshold
params_filename_prop8 = os.path.join(home, 'piecewise_params_prop8.joblib')
threshold_filename_prop8 = os.path.join(home, 'piecewise_threshold_prop8.joblib')


# Save the optimized parameters and threshold
# Make sure opt_params_8 and threshold_8 objects exist from a previous cell execution
if 'opt_params_8' in locals() and 'threshold_8' in locals():
    joblib.dump(opt_params_8, params_filename_prop8)
    joblib.dump(threshold_8, threshold_filename_prop8)
    print(f"Optimized parameters saved successfully to {params_filename_prop8}")
    print(f"Threshold saved successfully to {threshold_filename_prop8}")
else:
    print("Error: opt_params_8 or threshold_8 objects not found. Please run the cell that optimizes the piecewise function and defines the threshold first.")






Optimized parameters saved successfully to /content/drive/MyDrive/Projects/Shell/App/Models/piecewise_params_prop8.joblib
Threshold saved successfully to /content/drive/MyDrive/Projects/Shell/App/Models/piecewise_threshold_prop8.joblib


In [ ]:
# Create a dictionary to map property numbers to saved file names
saved_files_map = {
    1: {
        "model": 'linear_model_poly_target_1.joblib',
        "transform": 'poly1_features.joblib'
    },
    2: {
        "model": 'linear_model_poly_target_2.joblib',
        "transform": 'poly2_features.joblib'
    },
    5: {
        "model": 'tabpfn_model_target_5.tabpfn_fit',
        "transform": 'poly5_features.joblib'
    },
    6: {
        "model": 'linear_model_poly_target_6.joblib',
        "transform": 'poly6_features.joblib'
    },
    7: {
        "model": 'tabpfn_model_target_7.tabpfn_fit',
        # For Property 7, the transformation is the mixture feature generation,
        # which is not a saved object like PolynomialFeatures.
        # You would need to apply the generate_mixture_features function.
        "transform_function": "generate_mixture_features"
    },
    8: {
        # For Property 8, the "model" is the initial prediction model (not explicitly saved in this workflow)
        # and the correction is the piecewise function defined by parameters and threshold.
        "params": 'piecewise_params_prop8.joblib',
        "threshold": 'piecewise_threshold_prop8.joblib',
        "correction_function": "piecewise_model" # Reference the function name
    },
    10: {
        "model": 'linear_model_poly_target_10.joblib',
        "transform": 'poly10_features.joblib'
    }
}

# Display the dictionary
import json
print(json.dumps(saved_files_map, indent=4))